## 第一步：实现向量

行向量和列向量实际上都是向量，我们可以先定义一个向量的基类，然后把行向量和列向量作为其子类，定义其各自不同的性质。

我们注意到向量实际上可以看作为是一个一维数组，因此我们可以使用Python中的list来实现向量。但是为了封装一些方法，我们必须自己定义一个类。注意，我们建议使用**类型提示**，这样可以方便IDE进行代码提示。同时建议使用**Doc string**，这样可以方便IDE进行文档提示。

推荐在开发过程中，每个类都定义其`__repr__`方法，这样可以方便调试。

具体的细节参考Readme中的说明。

In [38]:
from typing import Self, Iterator


class Vector:
    """
    A basic Vector class

    Attributes:
        data (list[int | float | complex]): List containing the elements of the vector.

    Methods:
        __init__: Initializes the Vector object with the given elements.
        __getitem__: Returns the element at the specified index.
        __setitem__: Sets the element at the specified index to the given value.
        __len__: Returns the number of elements in the vector.
        __str__: Returns a string representation of the vector.
        __repr__: Returns a string representation of the vector that can be used to recreate the object.
        copy: Returns a copy of the vector.
        append: Appends a value to the end of the vector.
        pop: Removes and returns the last value of the vector.
        insert: Inserts a value at the specified index.
        remove: Removes and returns the value at the specified index.
    """

    data: list[int | float | complex]

    def __init__(self, *args: int | float | complex):
        self.data = list(args)

    def __getitem__(self, index: int):
        return self.data[index]

    def __setitem__(self, index: int, value: int | float | complex):
        self.data[index] = value

    def __len__(self):
        return len(self.data)

    def __str__(self) -> str:
        return ",".join([str(num) for num in self.data])

    def __repr__(self) -> str:
        return f"Vector({str(self)})"

    def copy(self) -> Self:
        """
        Return a copy of the vector
        """
        return type(self)(*self.data)

    def append(self, value: int | float | complex):
        """
        Append a value to the end of the vector

        :param value: the value to append
        :return: None
        """
        self.data.append(value)

    def pop(self) -> int | float | complex:
        """
        Pop the **last** value of the vector and return it

        :return: the last value
        """
        return self.data.pop()

    def insert(self, index: int, value: int | float | complex):
        """
        Insert a value to the vector at the given index

        :param index: the index where value is inserted
        :param value: the value to insert
        :return:  None
        """
        self.data.insert(index, value)

    def remove(self, index: int) -> int | float | complex:
        """
        Remove the value at the given index

        :param index: the index of the value to remove
        :return: the removed value
        """
        return self.data.pop(index)

    def __add__(self, other: 'Vector') -> Self:
        return type(self)(*[self[index] + other[index] for index in range(len(self))])
    # this is a little bit complicated. Create a list using a list comprehension, then use * to unpack it.
    # Use type(self) to create a new object of the same type as self, with arguments the elements of the list.

    def __sub__(self, other: 'Vector') -> Self:
        return type(self)(*[self[index] - other[index] for index in range(len(self))])
    
    def __iter__(self) -> Iterator[int | float | complex]:   # make the object iterable (for loop). In fact it is not necessary since a __getitem__ method is defined.
        return iter(self.data)

然后实现行向量和列向量。我们刚才在`copy()`函数中已经“预见到”了行向量和列向量的不同，因此我们直接用了`type(self)`去判断向量的类型。这样就不需要重写`copy()`函数了。其他同理。

In [39]:

class RowVector(Vector):
    """
    A basic RowVector class that inherits from Vector

    init method is not needed as it is the same as Vector:
    """

    def __str__(self) -> str:
        return "\t".join([str(_) for _ in self.data])

    def __repr__(self) -> str:
        return f"RowVector({super().__str__()})"

    def __mul__(self, other: 'ColumnVector') -> int | float | complex:
        if len(self) != len(other):
            raise ValueError("The length of the two vectors must be the same.")
        return sum(a * b for a, b in zip(self, other))  # Pylance says that we must add __iter__ to Vector base class but in fact not necessary. Anyway, make it happy.
    
    @property
    def matrix(self) -> 'Matrix':
        return Matrix(self)


class ColumnVector(Vector):
    """
    A basic ColumnVector class that inherits from Vector
    """

    def __str__(self) -> str:
        return "\n".join([str(_) for _ in self.data])

    def __repr__(self) -> str:
        return f"ColumnVector({super().__str__()})"
    
    @property
    def matrix(self) -> 'Matrix':
        return Matrix(*self.data, row_count=len(self), column_count=1)


In [40]:
# test
if __name__ == '__main__':
    rows: list[RowVector] = []
    columns: list[ColumnVector] = []
    with open('vector.csv', 'r', encoding='utf-8') as f:
        for i in f:
            line = i.strip().split(',')
            vec1 = RowVector(*map(eval, line))
            vec2 = ColumnVector(*map(eval, line))
            print(vec1)
            print(vec2)
            rows.append(vec1.copy())
            columns.append(vec2)
            vec1.remove(-1)
            vec2.append(0)
    print(rows)
    print(columns)

1	2	3	4	5
1
2
3
4
5
2	3	4	12	12.5
2
3
4
12
12.5
15	55	11	22	44
15
55
11
22
44
77	78	102	0	-114
77
78
102
0
-114
[RowVector(1,2,3,4,5), RowVector(2,3,4,12,12.5), RowVector(15,55,11,22,44), RowVector(77,78,102,0,-114)]
[ColumnVector(1,2,3,4,5,0), ColumnVector(2,3,4,12,12.5,0), ColumnVector(15,55,11,22,44,0), ColumnVector(77,78,102,0,-114,0)]


## 第二步：实现矩阵

有的人可能把矩阵直接作为一个二维数组来实现，但是这样占用资源会大。我们尝试用一个一维数组，然后用`__call__`实现行、列的访问——计算这个元素在一维数组中的位置，然后返回。

我们在`__init__`中加入了可选的参数`**kwargs`，并且允许直接传入若干个数字并规定行列，这样可以直接初始化一维数组；同时兼容传入若干个 `RowVector` 正如Readme中所说的，这样可以直接初始化矩阵。

In [69]:

from typing import Any, overload


class Matrix:
    """
    A basic Matrix class

    Args:
        *args (RowVector): Variable number of RowVector objects representing the rows of the matrix.

    Attributes:
        __data (list[int|float|complex]): List containing the elements of the matrix.
        __row_count (int): Number of rows in the matrix.
        __column_count (int): Number of columns in the matrix.

    Methods:
        count_rows: Returns the number of rows in the matrix.
        count_columns: Returns the number of columns in the matrix.
        get_columns: Returns a list of ColumnVector objects representing the specified columns.
        get_rows: Returns a list of RowVector objects representing the specified rows.
        insert_row: Inserts a row at the specified index.
        insert_column: Inserts a column at the specified index.
        remove_row: Removes the row at the specified index and returns it.
        remove_column: Removes the column at the specified index and returns it.
        copy: Returns a copy of the matrix.
        pop_row: Removes and returns the last row of the matrix.
        pop_column: Removes and returns the last column of the matrix.
        append_row: Appends a row to the end of the matrix.
        append_column: Appends a column to the end of the matrix.
        transpose: Returns the transpose of the matrix.
        rows: Returns a list of RowVector objects representing the rows of the matrix.
        columns: Returns a list of ColumnVector objects representing the columns of the matrix.
        __init__: Initializes the Matrix object with the given rows.
        __getitem__: Returns the element at the specified index.
        __setitem__: Sets the element at the specified index to the given value.
        __len__: Returns the total number of elements in the matrix.
        __call__: Returns the element at the specified index.
        __str__: Returns a string representation of the matrix.
        __repr__: Returns a string representation of the matrix that can be used to recreate the object.
        __add__: Adds two matrices.
        __sub__: Subtracts two matrices.
        __mul__: Multiplies two matrices.
    """
    __data: list[int | float | complex]  # It should not be accessed directly, otherwise the size of the matrix may be changed not meeting the row_count and column_count.
    __row_count: int = 0
    __column_count: int = 0

    @overload  # overload is used to define multiple signatures of a function. It is not necessary but can make the code more readable. Very useful in type hinting.
    def __init__(self, *args: RowVector) -> None: ...
    @overload
    def __init__(self, *args: int | float | complex, row_count: int, column_count: int) -> None: ...
    def __init__(self, *args: RowVector | int | float | complex, **kwargs: int):
        if not kwargs:
            self.__data: list[int | float | complex] = []
            self.__row_count = len(args)
            self.__column_count = 0
            for row in args:
                if not isinstance(row, RowVector):
                    raise TypeError("The arguments must be RowVector.")
                self.__data.extend(row.data)
                col_count = len(row.data)
                if col_count != self.__column_count:
                    if self.__column_count == 0:
                        self.__column_count = col_count
                    else:
                        raise ValueError("The number of columns in each row must be the same.")

        elif kwargs.keys() == {'row_count', 'column_count'}:
            # create directly using parameters row_count and column_count and *args
            self.__row_count = kwargs['row_count']
            self.__column_count = kwargs['column_count']
            self.__data = []
            # check if the number of elements is equal to row_count * column_count
            if len(args) != self.__row_count * self.__column_count:
                raise ValueError("The number of elements must be equal to row_count * column_count.")
            for elem in args:
                if not isinstance(elem, (int, float, complex)):
                    raise TypeError("The arguments must be int, float or complex.") # check types to avoid misusing
                self.__data.append(elem)
            
    def __getitem__(self, index: int) -> int | float | complex:
        return self.__data[index]

    def __setitem__(self, index: int, value: int | float | complex):
        self.__data[index] = value

    def __len__(self):
        return len(self.__data)

    def count_rows(self):
        """
        Returns the number of rows in the matrix.

        :return: int: The number of rows in the matrix.
        """
        return self.__row_count

    def count_columns(self):
        """
        Returns the number of columns in the matrix.

        :return: int: The number of columns in the matrix.
        """
        return self.__column_count

    def get_columns(self, *args: int):
        """
        Return a list of ColumnVector, each ColumnVector is a column of the matrix

        :param args: the indices of the columns
        :return: a list of ColumnVector
        """
        if not all(0 <= _ < self.__column_count for _ in args):
            raise IndexError('Invalid index')
        return [ColumnVector(*self.__data[_::self.__column_count]) for _ in args]

    def get_rows(self, *args: int) -> list[RowVector]:
        """
        Return a list of RowVector, each RowVector is a row of the matrix

        :param args: the indices of the rows
        :return: a list of RowVector
        """
        if not all(0 <= _ < self.__row_count for _ in args):
            raise IndexError('Invalid index')
        return [RowVector(*self.__data[ind * self.__column_count: (ind + 1) * self.__column_count]) for ind in args]

    def __str__(self) -> str:
        return "\n".join([str(row) for row in self.get_rows(*range(self.__row_count))])

    def __repr__(self) -> str:
        ret = ",".join([str(_) for _ in self.__data])
        return f"Matrix({ret}, row_count={self.__row_count}, column_count={self.__column_count})"

    def insert_row(self, index: int, row: RowVector) -> None:
        """
        Insert a row to the matrix at the given index

        :param index: the index where row is inserted
        :param row: the row to insert
        :return: None
        """
        if len(row) != self.__column_count:
            raise ValueError("The length of the row must be equal to the number of columns in the matrix.")
        if index > self.__row_count or index < 0:
            raise IndexError('Invalid index')
        left: list[int | float | complex] = self.__data[:index * self.__column_count]
        right: list[int | float | complex] = self.__data[index * self.__column_count:]
        self.__data = left + row.data + right
        self.__row_count += 1

    def insert_column(self, index: int, column: ColumnVector):
        """
        Insert a column to the matrix at the given index

        :param index: the index where column is inserted
        :param column: the column to insert
        :return: None
        """
        if len(column) != self.__row_count:
            raise ValueError("The length of the column must be equal to the number of rows in the matrix.")
        if index > self.__column_count or index < 0:
            raise IndexError('Invalid index')
        for index, value in zip(range(index, len(self.__data) + len(column.data), self.__row_count), column):
            self.__data.insert(index, value)
        self.__column_count += 1

    def remove_row(self, index: int) -> RowVector:
        """
        Remove the row at the given index

        :param index: the index of the row to remove
        :return: the removed row
        """
        if index >= self.__row_count or index < 0:
            raise IndexError('Invalid index')
        row = self.get_rows(index)
        left = self.__data[:index * self.__column_count]
        right = self.__data[(index + 1) * self.__column_count:]
        self.__data = left + right
        self.__row_count -= 1
        return row[0]

    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, Matrix):
            return False
        if len(self) != len(other):
            return False
        return all(self[index] == other[index] for index in range(len(self)))

    def remove_column(self, index: int) -> ColumnVector:
        """
        Remove the column at the given index

        :param index: the index of the column to remove
        :return: the removed column
        """
        if index >= self.__column_count or index < 0:
            raise IndexError('Invalid index')
        column = self.get_columns(index)[0]
        self.__column_count -= 1
        for i in range(self.count_rows()):
            self.__data.pop(index + i * self.__column_count)
        return column

    def copy(self):
        """
        Return a copy of the matrix
        """
        return Matrix(*self.__data, row_count=self.__row_count, column_count=self.__column_count)

    def pop_row(self):
        """
        Pop the **last** row of the matrix and return it

        :return: the last row
        """
        self.__row_count -= 1
        ret = self.get_rows(self.__row_count)[0]
        self.__data = self.__data[:-self.__column_count]
        return ret

    def pop_column(self) -> ColumnVector:
        """
        Pop the **last** column of the matrix and return it

        :return: the last column
        """
        self.__column_count -= 1
        ret = self.get_columns(self.__column_count)[0]
        for index in range(self.__column_count, len(self.__data) - self.__row_count, self.__column_count):
            self.__data.pop(index)
        return ret

    def append_row(self, row: RowVector) -> None:
        """
        Append a row to the end of the matrix

        :param row: the row to append
        :return: None
        """
        if len(row) != self.__column_count:
            raise ValueError("The length of the row must be equal to the number of columns in the matrix.")
        self.__data.extend(row.data)
        self.__row_count += 1

    def append_column(self, column: ColumnVector) -> None:
        """
        Append a column to the end of the matrix

        :param column: the column to append
        :return: None
        """
        if len(column) != self.__row_count:
            raise ValueError("The length of the column must be equal to the number of rows in the matrix.")
        column_generator = iter(column)
        for index in range(self.__column_count, len(self.__data) + len(column.data), self.__row_count):
            self.__data.insert(index, next(column_generator))
            # using zip is also OK, but you should learn how to use iter and next.
            # You can refer to function insert_column, as they are similar.
        self.__column_count += 1

    def __add__(self, other: 'Matrix') -> 'Matrix':
        if self.count_rows() != other.count_rows() or self.count_columns() != other.count_columns():
            raise ValueError("The size of the two matrices must be the same.")
        ret = self.copy()
        for index in range(len(self)):
            ret[index] += other[index]
        return ret

    def __sub__(self, other: 'Matrix') -> 'Matrix':
        if self.count_rows() != other.count_rows() or self.count_columns() != other.count_columns():
            raise ValueError("The size of the two matrices must be the same.")
        ret = self.copy()
        for index in range(len(self)):
            ret[index] -= other[index]
        return ret

    def __mul__(self, other: 'Matrix') -> 'Matrix':
        if self.count_columns() != other.count_rows():
            raise ValueError(
                "The number of columns in the first matrix must be equal to the number of rows in the second matrix.")
        return Matrix(*(
            sum(map(lambda x, y: x * y, row, col)) for row in (
                    self.__data[
                        index: index + self.count_columns()
                    ] for index in range(0, len(self), self.count_columns())
                ) for col in (
                    other.__data[index: len(other): other.count_columns()] for index in range(other.count_columns())
                )
            ),
                    row_count=self.count_rows(),
                    column_count=other.count_columns(),
                )

    @property  # make a function a property, so that it can be called without parentheses.
    def rows(self) -> list[RowVector]:
        """
        Return a list of RowVector, each RowVector is a row of the matrix

        :return: a list of RowVector
        """
        return self.get_rows(*range(self.__row_count))

    @property
    def columns(self) -> list[ColumnVector]:
        """
        Return a list of ColumnVector, each ColumnVector is a column of the matrix

        :return: a list of ColumnVector
        """
        return self.get_columns(*range(self.__column_count))

    def __call__(self, x: int, y: int) -> int | float | complex:
        if x >= self.__row_count or y >= self.__column_count or x < 0 or y < 0:
            raise IndexError("Index out of range.")
        return self.__data[x * self.__column_count + y]

    def transpose(self, operate_on_self: bool = False, copy: bool = False) -> 'Matrix':
        """
        Return the transpose of the matrix

        :param operate_on_self: whether to operate directly on self or return a new matrix (default: False)
        :param copy: whether to return a copy or just self if operate_on_self is True (default: False)
        :return: the transpose of the matrix
        """
        if not operate_on_self:
            return self.copy().transpose(operate_on_self=True)
        new_data: list[int | float | complex] = []
        z = zip(*self.rows)
        for row in z:
            new_data.extend(row)
        self.__data = new_data
        self.__row_count, self.__column_count = self.__column_count, self.__row_count
        return self.copy() if copy else self

    def left_multiply(self, other:'Matrix') -> 'Matrix':
        return other * self

    def right_multiply(self, other:'Matrix') -> 'Matrix':
        return self * other

In [70]:
# test
if __name__ == '__main__':
    mat = Matrix(*rows)  # here you need to make a Matrix from the list of RowVectors
    print(mat)
    print(mat[0])
    print(len(mat))
    print(mat.count_rows())
    print(mat.count_columns())
    print(mat.get_columns(0, 1))
    print(mat.get_rows(0, 1))
    print(f"to insert two rows:\n{rows[0]}\n{rows[1]}")
    mat.insert_row(0, rows[0])
    mat.append_row(rows[1])
    print(mat(0, 2))
    print(f"to insert a column:\n{columns[0]}")
    mat.insert_column(0, columns[0])
    mat.remove_column(5)
    print(mat.copy())

1	1	2	3	4
2	1	2	3	4
3	2	3	4	12
4	15	55	11	22
5	77	78	102	0
0	2	3	4	12


## 第三步：实现向量的加减

这部分内容已经在向量的类里面，通过定义了`__add__`和`__sub__`方法来实现。这样我们就可以直接使用`+`和`-`来进行向量的加减。

## 第四步：实现矩阵的基本运算

在上面其实我们已经定义好了运算符号，使用的是对应的**魔法方法**。

但是，这些运算符或许还是存在一些限制（主要是已有的类型）。我们可以定义函数，更加自由一些。

鉴于“如无必要，勿增实体”，我们此处不再重写`add`、`sub`。

下面我们实现乘法——数乘、矩阵乘法、向量点乘。

先实现数乘：

In [ ]:
@overload
def scalar_multiply(a: Matrix, num: int | float | complex) -> Matrix: ...


@overload
def scalar_multiply(a: RowVector, num: int | float | complex) -> RowVector: ...


@overload
def scalar_multiply(a: ColumnVector, num: int | float | complex) -> ColumnVector: ...


@overload
def scalar_multiply(a: Vector, num: int | float | complex) -> Vector: ...


@overload
def scalar_multiply(a: Any, num: int | float | complex) -> Any: ...


def scalar_multiply(a: Matrix | RowVector | ColumnVector | Vector | Any,
                    num: int | float | complex) -> Matrix | RowVector | ColumnVector | Vector | Any:
    """
    Multiply a vector or matrix by a scalar

    :param a: the vector or matrix
    :param num: the scalar
    :return: the product
    """
    if isinstance(a, Matrix):
        return Matrix(*(num * elem for elem in a), row_count=a.count_rows(), column_count=a.count_columns())
    elif isinstance(a, (RowVector, ColumnVector, Vector)):
        return type(a)(*[num * num for num in a])
    else:
        return a * num

然后是乘法（调用`__mul__`）：

In [ ]:
def mul(a: Matrix | RowVector | ColumnVector | Any, b: Matrix | RowVector | ColumnVector | Any) -> Matrix | Any:
    """
    Multiply two vectors or matrices or other objects

    :param a: the first vector or matrix or something else
    :param b: the second vector or matrix or something else
    :return: the product
    """
    if isinstance(a, (int, float, complex)):
        return scalar_multiply(b, a)
    if isinstance(b, (int, float, complex)):
        return scalar_multiply(a, b)
    match a:
        case Matrix():
            match b:
                case Matrix():
                    return a * b
                case RowVector() | ColumnVector():
                    return a * b.matrix
                case _:
                    raise TypeError("The second argument must be Matrix, RowVector , ColumnVector or scalar when the "
                                    "first argument is Matrix.")
        case RowVector():
            match b:
                case Matrix():
                    return a.matrix * b
                case ColumnVector():
                    return a * b
                case _:
                    raise TypeError("The second argument must be Matrix, RowVector , ColumnVector or scalar when the "
                                    "first argument is RowVector.")
        case ColumnVector():
            match b:
                case Matrix():
                    return a.matrix * b
                case RowVector():
                    return a.matrix * b.matrix
                case _:
                    raise TypeError("The second argument must be Matrix, RowVector , ColumnVector or scalar when the "
                                    "first argument is ColumnVector.")
        case _:
            return a * b

In [ ]:
def add(a,b):
    return a+b
def subtract(a,b):
    return a-b



In [71]:
# test
if __name__ == '__main__':
    print(add('---', '---'))
    print(add(mat, mat))
    print(mat.transpose(operate_on_self=True, copy=True))
    print(mul('-', 15))
    print(mat.transpose(operate_on_self=True, copy=False))
    print(add(mat, mat))
    print(subtract(mat, mat.copy()))
    print(scalar_multiply(mat, 2))
    print(mat * mat.transpose())
    print(mat.transpose() * mat)
    

------
2	2	4	6	8
4	2	4	6	8
6	4	6	8	24
8	30	110	22	44
10	154	156	204	0
0	4	6	8	24
1	2	3	4	5	0
1	1	2	15	77	2
2	2	3	55	78	3
3	3	4	11	102	4
4	4	12	22	0	12
---------------
1	1	2	3	4
2	1	2	3	4
3	2	3	4	12
4	15	55	11	22
5	77	78	102	0
0	2	3	4	12
2	2	4	6	8
4	2	4	6	8
6	4	6	8	24
8	30	110	22	44
10	154	156	204	0
0	4	6	8	24
0	0	0	0	0
0	0	0	0	0
0	0	0	0	0
0	0	0	0	0
0	0	0	0	0
0	0	0	0	0
2	2	4	6	8
4	2	4	6	8
6	4	6	8	24
8	30	110	22	44
10	154	156	204	0
0	4	6	8	24
31	32	71	250	544	68
32	34	74	254	549	68
71	74	182	515	811	173
250	254	515	3871	6587	503
544	549	811	6587	22442	796
68	68	173	503	796	173
55	454	625	575	136
454	6164	6847	8041	386
625	6847	9135	8597	1298
575	8041	8597	10575	362
136	386	1298	362	804


## 第五步

那么，我们就拿“行列式”举例吧，剩下的事情我也不多说了。

In [79]:
# 计算行列式
def det(mat: Matrix) -> int | float | complex:
    """
    Calculate the determinant of a matrix

    :param mat: the matrix
    :return: the determinant
    """
    if mat.count_rows() != mat.count_columns():
        raise ValueError("The matrix must be square.")
    if mat.count_rows() == 1:
        return mat(0, 0)
    ret = 0
    for index in range(mat.count_columns()):
        submat = mat.copy()
        submat.remove_row(0)
        submat.remove_column(index)
        ret += (-1) ** index * mat(0, index) * det(submat)  # this is called recursion
    return ret

In [81]:
# test

if __name__ == '__main__':
    mat = Matrix(10000,2,3,4,5,6,7,8,9,10,11,12,row_count=3,column_count=4)
    try:
        print(det(mat))
    except ValueError as e:
        mat.append_row(RowVector(13,14,15,16))
        print(det(mat))

0
